Determine the transformation of an aperture diameter of 7.5 kpc at the redshift of abell 370 to pixels. First transform the diameter in absolute size (kpc) to apparent size (arcseconds). Then transform apparent size to pixels using WCS.

In [1]:
import numpy as np 

from astropy.io import fits
from astropy import units as u 
from astropy import cosmology
from astropy.wcs import WCS

### Load Data

In [2]:
input_data_path = "https://archive.stsci.edu/pub/hlsp/frontier/abell370/images/hst/v1.0-epoch2/hlsp_frontier_hst_wfc3-30mas-bkgdcor_abell370_f105w_v1.0-epoch2_drz.fits"

hdul = fits.open(input_data_path)
hdu = hdul[0]

data = hdu.data
header = hdu.header
wcs = WCS(header)

### Given

In [3]:
# Given 
aperture_diameter = (7.5 * u.kpc).to(u.Mpc)

NED data for abell 370:

http://ned.ipac.caltech.edu/cgi-bin/objsearch?search_type=Obj_id&objid=132527&objname=1&img_stamp=YES&hconst=73.0&omegam=0.27&omegav=0.73&corr_z=1

In [31]:
abell370_z = 0.375000
abell370_distance = 1413 * u.Mpc # Comoving

### Equations

\begin{align}
\theta = \frac{(z+1) * Diameter}{Distance_{Comoving}} =  \frac{Diameter}{Distance_{Angular Diameter} } \\
\end{align}

### Compute angular diameter distance

In [32]:
def compute_angular_diameter_distance(d, z):
    return d/(z+1)

In [33]:
angular_diameter_distance = compute_angular_diameter_distance(abell370_distance, abell370_z)
angular_diameter_distance

<Quantity 1027.63636364 Mpc>

In [39]:
# check astropy cosmology
cosmology.WMAP5.angular_diameter_distance(abell370_z)

<Quantity 1068.40525789 Mpc>

### Find angular diameter

In [7]:
angular_diameter = np.tan((aperture_diameter / angular_diameter_distance).value) * u.rad 
angular_diameter 

<Quantity 5.20176545e-06 rad>

In [42]:
# check astropy cosmology
(cosmology.WMAP5.arcsec_per_kpc_comoving(abell370_z) * aperture_diameter.to('kpc')).to('rad')

<Quantity 5.10531506e-06 rad>

### Convert to arcsec

In [8]:
angular_diameter = angular_diameter.to("arcsec")
angular_diameter

<Quantity 1.07294114 arcsec>

### Find Pixel Size 

In this section we convert the angular size into pixel size. The way we do this is by adding the angular size to the `CRVAL` of the WCS and then converting that world coordinate to pixel values. After getting the new pixel values we subtract the `CRPIX` values to find the difference (angular size). Because RA-DEC is an equatorial coordinate system, I have decided its best to add the angular size to the Dec component of the WCS CRVAL (Dec axis is a great circle). It looks like the image `CRVAL` is sufficiently away from the pols of the celestial sphere, so no need to worry about loops around +/- 90 degrees.  

In [15]:
# Load center pixel from data's WCS 

ra_0 = wcs.wcs.crval[0] * u.Unit(wcs.wcs.cunit[0])
dec_0 = wcs.wcs.crval[1] * u.Unit(wcs.wcs.cunit[1])

print(ra_0, ",",  dec_0)

39.96301 deg , -1.5882933 deg


In [19]:
# Add angular_diameter to center Dec value 

ra_1  = ra_0
dec_1 = dec_0 + angular_diameter

print(ra_1, ",",  dec_1)

39.96301 deg , -1.587995260793829 deg


In [20]:
# Convert ra_1, dec_1 into pixels

world = np.array([[ra_1.value, dec_1.value],], dtype=np.float64) * u.deg
pixcrd = wcs.wcs_world2pix(world, 1)

pixcrd

array([[4800.        , 6635.76470474]])

In [24]:
# Look at the center pixel values
wcs.wcs.crpix

array([4800., 6600.])

In [25]:
# Find the difference b/w the center pixel and the angular offset pixel:
pixel_diff =  pixcrd - wcs.wcs.crpix 
assert abs(pixel_diff[0,0]) < 1e-10
pixel_diff[0,1]

35.76470474127291

In [26]:
# Find ceiling of pixel diff to conclude pixel size
pixel_size = int(np.ceil(pixel_diff[0,1]))

pixel_size

36